# Road Following - Live demo (TensorRT)

In this notebook, we will use model we trained to move JetBot smoothly on track. 

# TensorRT

In [28]:
import torch
device = torch.device('cuda')

Load the TRT optimized model by executing the cell below

In [29]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth'))

<All keys matched successfully>

### Creating the Pre-Processing Function

We have now loaded our model, but there's a slight issue. The format that we trained our model doesn't exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [30]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera. You should be pretty familiar with this by now. 

In [31]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera
from jetbot import bgr8_to_jpeg



In [32]:
camera = Camera.instance(width=224,height=224,fps=10)


In [33]:
image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

We'll also create our robot instance which we'll need to drive the motors.

In [34]:
from jetbot import Robot

robot = Robot()

Now, we will define sliders to control JetBot
> Note: We have initialize the slider values for best known configurations, however these might not work for your dataset, therefore please increase or decrease the sliders according to your setup and environment

1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
2. Steering Gain Control (steering_gain_slider): If you see JetBot is wobbling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

In [35]:
# speed_gain_slider = ipywidgets.FloatSlider(min=0.25, max=1.0, step=0.01, description='speed gain')
# steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='steering gain')
# steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
# steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

speed gain: 0.13
<br>steering gain: 0.03
<br>steering kd: 0.02
<br>steering bias: 0.00

Next, let's display some sliders that will let us see what JetBot is thinking.  The x and y sliders will display the predicted x, y values.

The steering slider will display our estimated steering value.  Please remember, this value isn't the actual angle of the target, but simply a value that is
nearly proportional.  When the actual angle is ``0``, this will be zero, and it will increase / decrease with the actual angle.  

In [36]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. Compute the approximate steering value
4. Control the motors using proportional / derivative control (PD)

In [37]:

# angle = 0.0
# angle_last = 0.0
# count_motor = 0
# def execute(change):
#     global angle, angle_last, count_motor
#     image = change['new']
#     xy = model_trt(preprocess(image)).detach().float().cpu().numpy().flatten()
#     x = xy[0]
#     y = (0.5 - xy[1]) / 2.0
    
#     x_slider.value = x
#     y_slider.value = y
    
#     speed_slider.value = speed_gain_slider.value
    
#     angle = np.arctan2(x, y)
#     pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
#     angle_last = angle
    
#     steering_slider.value = pid + steering_bias_slider.value
   
#     if count_motor >2:
#         robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0)*0.95, 0.0)
#         robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
#         count_motor = 0
#     else :
#         robot.left_motor.value = 0
#         robot.right_motor.value = 0
#         count_motor = count_motor + 1

        
# execute({'new': camera.value})
angle = 0.0
angle_last = 0.0


def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model_trt(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
#     y = 0.5 - xy[1] / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
#     print( x, y, angle)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

-0.07867432 0.084228515625 -0.751316199115295


Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower should work, but the neural network is only as good as the data it's trained on!

In [38]:
camera.observe(execute, names='value')


-0.06750488 0.08251953125 -0.6856489502039712
-0.06298828 0.08544921875 -0.6352216291707345
-0.06842041 0.08642578125 -0.6696389456766376
-0.066223145 0.087890625 -0.6457196298346324
-0.06994629 0.08544921875 -0.6859628468280063
-0.068603516 0.08447265625 -0.6820989033383003
-0.0692749 0.087158203125 -0.6715731446006387
-0.076293945 0.086181640625 -0.7246166256438834
-0.07495117 0.087646484375 -0.7074781670064094
-0.07336426 0.089599609375 -0.686100348240025
-0.07434082 0.089599609375 -0.6925903401792551
-0.07611084 0.08740234375 -0.7164520006361452
-0.0713501 0.087158203125 -0.6859963910707534
-0.07208252 0.0908203125 -0.6708771008795974
-0.076171875 0.089599609375 -0.7045731531379719
-0.072387695 0.090576171875 -0.6742476834671081
-0.06695557 0.0927734375 -0.6251486715918636
-0.07525635 0.08837890625 -0.7053754566142888
-0.07159424 0.089599609375 -0.6741591603742977
-0.07385254 0.08935546875 -0.6906935362208899
-0.070495605 0.0888671875 -0.6706233184360848
-0.068481445 0.088134765625

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow track.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [39]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

Again, let's close the camera conneciton properly so that we can use the camera in other notebooks.

In [40]:
camera.stop()

### Conclusion
That's it for this live demo! Hopefully you had some fun seeing your JetBot moving smoothly on track following the road!!!

If your JetBot wasn't following road very well, try to spot where it fails. The beauty is that we can collect more data for these failure scenarios and the JetBot should get even better :)